In [1]:
import os
import numpy as np
import re

In [2]:
idx_audio = np.hstack([np.arange(1, 9), np.arange(10, 16), np.arange(18, 21)]).tolist()
idx_seeg = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 19, 20, 21]

In [3]:
# load greenbook_01.wav to greenbook_19.wav and concatenate them into numpy array
import soundfile as sf
import numpy as np

filenames = ["e0010GP_20201118/xmen/xmen_{:02d}.wav".format(i) for i in range(1, 25)]
concatenated_audio = []
# I need to have a list counting the start time and end time of each audio file
# I will use the following list for the purpose
audio_time = []
start_time = 0

for filename in filenames:
    audio, samplerate = sf.read(filename)
    audio = np.mean(audio, axis=1)
    concatenated_audio.append(audio)
    audio_time.append([start_time, start_time + len(audio)/samplerate])
    start_time += len(audio)/samplerate

concatenated_audio = np.concatenate(concatenated_audio)
audio_time = np.array(audio_time)


In [4]:
audio_time[idx_audio]
true_start_time = [0]
for idx in idx_audio:
    true_start_time.append(audio_time[idx][1] - audio_time[idx][0])
true_start_time = np.cumsum(true_start_time)[:-1]

In [5]:
with open("XMen.srt", "r") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
segments = []
for line in lines:
    if '-->' in line:
        time_matches = re.findall(r'\d{2}:\d{2}:\d{2},\d{3}', line)
        if len(time_matches) == 2:
            h1, m1, s1 = map(float, time_matches[0].replace(',', '.').split(':'))
            start_time = h1 * 3600.0 + m1 * 60.0 + s1 + 0.20
            h2, m2, s2 = map(float, time_matches[1].replace(',', '.').split(':'))
            end_time = h2 * 3600.0 + m2 * 60.0 + s2 + 0.20

            flag = False
            for idx in range(len(audio_time)):
                if ((audio_time[idx][0] <= start_time <= audio_time[idx][1]) and (audio_time[idx][0] <= end_time <= audio_time[idx][1])):
                    flag = True
                    break
            if flag == False:
                continue
            if idx not in idx_audio:
                continue
            else:
                start_time = start_time - audio_time[idx][0] + true_start_time[idx_audio.index(idx)]
                end_time = end_time - audio_time[idx][0] + true_start_time[idx_audio.index(idx)]
                segments.append((start_time, end_time))

In [6]:
audio_segments = []
for segment in segments:
    start_time, end_time = segment
    start_index = int(start_time * samplerate)
    end_index = int(end_time * samplerate)
    audio_segment = concatenated_audio[start_index:end_index]
    audio_segments.append(audio_segment)    

In [7]:
seeg_contacts = np.load("e0010GP_20201118_seeg_contacts.npy")
seeg_sr = 1024.0
seeg_segments = []
for segment in segments:
    start_time, end_time = segment
    start_index = int(start_time * seeg_sr)
    end_index = int(end_time * seeg_sr)
    seeg_segment = seeg_contacts[:, start_index:end_index]
    seeg_segments.append(seeg_segment)


In [8]:
np.save('e0010GP_20201118_data_segmented.npy', {'audio': audio_segments, 'seeg': seeg_segments})

In [17]:
temp = np.load('e0006KR_data_segmented.npy', allow_pickle=True).item()
audio_segments = temp['audio']
seeg_segments = temp['seeg']
print(len(seeg_segments))

1788


In [14]:
# # play the audio of audio_segments[21]
# import sounddevice as sd
# sd.play(audio_segments[251], samplerate)